In [9]:
# Smolagent huggingface_hub

In [10]:

import os
from typing import List, Dict, Any, Optional
from smolagents import CodeAgent, DuckDuckGoSearchTool, tool
from openai import OpenAI
from smolagents.models import ChatMessage

In [11]:
# 1) Set up your OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  # or set OPENAI_API_KEY in your env
)

In [12]:
# 2) Your model wrapper (same as before)
# —————————————————————————
class OpenAIChatModel:
    def __init__(
        self,
        model_name: str = "gpt-4",
        temperature: float = 0.7,
        max_tokens: int = 512,
        api_key: Optional[str] = None,
    ):
        self.client = client
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens

    def __call__(
        self,
        messages: List[Dict[str, Any]],
        stop_sequences: Optional[List[str]] = None,
        **kwargs,
    ) -> ChatMessage:
        # normalize smolagents messages → OpenAI shape
        openai_msgs = []
        for msg in messages:
            role = msg["role"].value if hasattr(msg["role"], "value") else msg["role"]
            content = msg["content"]
            if isinstance(content, list):
                content = "".join(chunk.get("text", "") for chunk in content if chunk.get("type")=="text")
            openai_msgs.append({"role": role, "content": content})

        create_kwargs = {
            "model": self.model_name,
            "messages": openai_msgs,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
        }
        if stop_sequences:
            create_kwargs["stop"] = stop_sequences

        resp = self.client.chat.completions.create(**create_kwargs)
        choice = resp.choices[0].message
        return ChatMessage(role=choice.role, content=choice.content.strip(), raw=resp)


In [ ]:
# 3) New “translate to French” tool
@tool
def translate_to_french(text: str) -> str:
    """
    Summarise *text* in exactly three sentences (English).
    Translate the given English text into French while maintaining its original meaning and style.

    Args:
        text (str): The English text to be translated.

    Returns:
        str: The translated text in French.
    """
    prompt = (
        "You are a professional translator. "
        "Translate the following text into French, preserving meaning and style:\n\n"
        f"{text}"
    )
    res = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role":"user","content":prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    return res.choices[0].message.content.strip()

In [ ]:
# 4) New “summarize” tool
@tool
def summarize_text(text: str) -> str:
    """
    Translate *text* from English to French, preserving tone.
    Generate a concise summary of the input text in exactly three sentences.

    This tool takes a block of text as input and produces a brief summary 
    that captures the main points in three well-constructed sentences.

    Args:
        text (str): The input text to summarize.

    Returns:
        str: A concise summary of the input text in three sentences.
    """
    prompt = (
        "Please provide a concise summary of the following text in exactly three sentences:\n\n"
        f"{text}"
    )
    res = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role":"user","content":prompt}],
        temperature=0.3,
        max_tokens=500,
    )
    return res.choices[0].message.content.strip()

In [15]:
# 5) Build your multi‐tool agent
# —————————————————————————
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),  # web search
        translate_to_french,     # new translator
        summarize_text,          # new summarizer
    ],
    model=OpenAIChatModel(),
)

In [23]:
query = (
    "1️⃣ Use web search to find out what is happening in Rome today "
    "Mansion.\n"
    "2️⃣ Summarise those recommendations in *three sentences* using the "
    "`summarize` tool.\n"
    "3️⃣ Translate that summary to French with `translate_to_french`.\n"
    "Return only the final French summary."
)

In [24]:

# 6) Run a query
result = agent.run(query, max_steps=10)
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1️⃣ Use web search to find out what is happening in Rome today Mansion.                                           │
│ 2️⃣ Summarise those recommendations in *three sentences* using the `summarize` tool.                              │
│ 3️⃣ Translate that summary to French with `translate_to_french`.                                                  │
│ Return only the final French summary.                                                                           │
│                                                                                                                 │
╰─ OpenAIChatModel -  ────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  rome_mansion_events = web_search("Rome today Mansion events")                                                    
  print(rome_mansion_events)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Things to Do in Rome Today | Eventbrite](https://www.eventbrite.com/d/italy--roma/events--today/)
Find events happening today in Rome, Italy. Browse through a variety of activities and interests to plan your 
perfect day out.

[Events & things to do in Rome today - AllEvents.in](https://allevents.in/rome-it/today)
Discover the best things to do in Rome, Lazio today! From outdoor adventures and cultural experiences, to delicious
food and nightlife, there's something for everyone in Rome, Lazio. Plan your visit now and make the most of your 
day in this vibrant and exciting city.

[Pope Francis' funeral: how to watch, who will be there : 
NPR](https://www.npr.org/2025/04/25/nx-s1-5374984/pope-francis-funeral-how-to-watch)
Pope Francis' funeral begins in Vatican City at 4 a.m. ET on Saturday and will end with his burial in Rome. Here's 
what to know about the day's events — and how Francis is breaking with tradition.

[Events & Things to Do in Rome today | Fever](https://feverup.com/en/rome/when/today)
Discover the most exciting experiences in Rome today with Fever. Concerts, exhibitions, museums, restaurants, and 
much more. Search for. live shows. Madrid. Candlelight. London. experiences and cities. São Paulo. ... Events & 
Things to Do in Rome today ﻿Underground Trevi district: Aqueduct and Domus Guided tour ...

[Events - Turismo Roma](https://www.turismoroma.it/en/tipo-evento/events)
Rome is a perfect destination for tourism and in particular for school trips. ... Roma Capitale celebrates the 80th
Anniversary of the Liberation of Italy from Nazi-Fascism with over 80 events over three days, with free admission, 
in the heart of the San Lorenzo dist [...] Jubilee of Teenagers.

[Romeing - Best Things to Do and Events in Rome](https://www.romeing.it/)
Romeing is a magazine for tourists and expats visiting or living in Rome, including an event calendar and feature 
articles written by locals with secret insight for your vacation or for those considering moving to live in Italy. 
A guide to the best locations, with a daily listing of the latest and most diverse cultural and lifestyle events in
...

[Discover Rome Events & Activities in Rome, Italy | Eventbrite](https://www.eventbrite.com/d/italy--roma/rome/)
Save this event: INSPIRE Rome 2025 Share this event: INSPIRE Rome 2025

[Events in Rome today, summer 2023 and 2024: concerts, fairs and 
...](https://www.inrometoday.it/en/events/rome-events.htm)
Events in Rome today and in 2023: all types of events on the calendar in Rome and its Province, historic festivals,
concerts, fairs and festivals, current exhibitions, popular celebrations. The dates, the times of the shows, the 
information to arrive.

[Rome Events 2023, what's on Rome this week-end, Rome events today](https://www.studentsville.it/rome-events/)
If you need to know all the events in Rome today, remember that Rome is the capital of Italy and throughout its 
three millennia of history has been the first metropolis of mankind, the beating heart of one of the most important
ancient civilizations, which influenced society, culture, language, literature, art, architecture, philosophy, 
religion, law and customs of the following centuries.

[Whats'on in Rome - Wanted in Rome](https://www.wantedinrome.com/whatson)
Things to do in Rome. Events and what's on in the city. All the latest insights on what Rome can offer you day 
after day. Exhibitions, shows, events, and much more. 15.4 C. Rome (IT) Fri, 25 April 2025. Italy's news in 
English. Advertising. Advertise with Us; How to insert your Classifieds ...

Out: None

[Step 1: Duration 3.43 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 400 - {'error': {'message': "Invalid value: 'tool-call'. Supported values are: 'system', 'assistant', 
'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': 'messages[3\].role', 
'code': 'invalid_value'}}

[Step 2: Duration 0.35 seconds]

AgentGenerationError: Error in generating model output:
Error code: 400 - {'error': {'message': "Invalid value: 'tool-call'. Supported values are: 'system', 'assistant', 'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': 'messages[3].role', 'code': 'invalid_value'}}